In [5]:
import gurobipy as gb
from gurobipy import *
import numpy as np

# Rummikub

Rummikub is a game that combines elements of classic card games like Rummy with the strategy of tile placement. The game is played with a set of 106 tiles, with numbers ranging from 1 to 13 in four different colors (red, blue, yellow, and black). Additionally, there are two joker tiles in the set. The game is typically played by 2 to 4 players and revolves around the strategic placement and manipulation of numbered tiles. The objective of Rummikub is to be the first player to empty your rack of tiles by forming sets and runs of matching numbers. Sets consist of three or four tiles of the same number but different colors. For example, you could have a set of 3s with one red, one blue, and one black. Runs are sequences of at least three consecutive numbers of the same color. For instance, you could have a run of 4, 5, 6 in blue. The game continues until one player goes out, at which point they gain opponents' tile values, while others receive penalties determined by the remaining tiles in their racks.

In this project, our primary objective is to address Rummikub challenges through the application of integer linear programming. Initially, we plan to focus on a two-player scenario, with the potential to expand to a four-player format if time permits. Also, if time allows, our ultimate goal is to develop an interactive online Rummikub board game, providing users with a platform for engaging gameplay.

## Set up
$2*4*13 + joker*2$

In [6]:
Deck = {"Black": ["1", "1", "2", "2", "3", "3", "4", "4", "5", "5", "6", "6", 
                  "7", "7", "8", "8", "9", "9", "10", "10", "11", "11", "12", "12", "13", "13"], 
        "Red": ["1", "1", "2", "2", "3", "3", "4", "4", "5", "5", "6", "6", 
                  "7", "7", "8", "8", "9", "9", "10", "10", "11", "11", "12", "12", "13", "13"], 
        "Blue": ["1", "1", "2", "2", "3", "3", "4", "4", "5", "5", "6", "6", 
                  "7", "7", "8", "8", "9", "9", "10", "10", "11", "11", "12", "12", "13", "13"], 
        "Orange": ["1", "1", "2", "2", "3", "3", "4", "4", "5", "5", "6", "6", 
                  "7", "7", "8", "8", "9", "9", "10", "10", "11", "11", "12", "12", "13", "13"], 
        "Joker": ["J1", "J2"]}

In [7]:
Penalty = {"1": 1, "2":2, "3":3, "4":4, "5":5, "6":6, "7":7, "8":8, "9": 9, "10":10, "11":11, "12":12, 
           "13":13, "Joker": 30}

In [8]:
Value = [1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11, 11, 12, 12, 13, 13]
Value_matrix = Value * 4

In [9]:
model = gb.Model("Rummikub")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-30


In [10]:
Deck_types = ["Black1", "Black2", "Red1", "Red2", "Blue1", "Blue2", "Orange1", "Orange2"]
# Deck_types = ["Black", "Red", "Orange", "Blue"]
Deck_values = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
Joker = ["Joker"]
Joker_values = ["J1", "J2"]

In [67]:
# I = len(Value_matrix)
J = 1173

#### All Possible Sets

In [18]:
# Adjusting the setup to consider only the deck color
Deck_colors = ["Black", "Red", "Blue", "Orange"]

# Generating all possible sets with three consecutive numbers
# and represent each card in the set using Deck_color and Deck_values
three_consecutive_set_no_joker = []

for value in Deck_values[:-2]:  # Iterate through the values, stopping two before the end
    for color in Deck_colors:
        # Create a set of three consecutive cards of the same color
        consecutive_set = [(color, value), (color, value + 1), (color, value + 2)]
        three_consecutive_set_no_joker.append(consecutive_set)


In [19]:
# Adjusting the setup to include one Joker in each set
# The Joker can replace any one of the three cards in the set

three_consecutive_set_one_joker = []
# Iterate through each color and value, creating sets with one Joker
for color in Deck_colors:
    for value in Deck_values[:-2]:  # Iterate through the values, stopping two before the end
        # For each set of three consecutive values, create three sets, each with one Joker
        if value == 1:
            for i in range(3):
                consecutive_set_one_joker = [(color, value), (color, value + 1), (color, value + 2)]
                consecutive_set_one_joker[i] = "Joker"  # Replace one card with a Joker
                three_consecutive_set_one_joker.append(consecutive_set_one_joker)
        else:
            for i in range(2):
                consecutive_set_one_joker = [(color, value), (color, value + 1), (color, value + 2)]
                consecutive_set_one_joker[i] = "Joker"  # Replace one card with a Joker
                three_consecutive_set_one_joker.append(consecutive_set_one_joker)


In [20]:
# Adjusting the setup to include exactly two Jokers in each set
# Each set will now consist of one card from the deck and two Jokers

# Generating all possible sets with one card from the deck and two Jokers
three_consecutive_sets_with_two_jokers = []

for color in Deck_colors:
    for value in Deck_values:  # Iterate through all values
        # Create a set with one card from the deck and two Jokers
        set_with_two_jokers = [(color, value), "Joker", "Joker"]
        three_consecutive_sets_with_two_jokers.append(set_with_two_jokers)

In [21]:
# Generating sets with four consecutive numbers, same color, and without Joker

four_consecutive_sets_no_joker = []

for color in Deck_colors:
    for value in Deck_values[:-3]:  # Iterate through values, stopping three before the end
        # Create a set of four consecutive cards of the same color
        consecutive_set = [(color, value), (color, value + 1), (color, value + 2), (color, value + 3)]
        four_consecutive_sets_no_joker.append(consecutive_set)

In [22]:
four_consecutive_sets_one_joker = []

for color in Deck_colors:
    for value in Deck_values[:-3]:  # Iterate through values, stopping three before the end
        # Create a set of four consecutive cards of the same color
        if value == 1:
            for i in range(4):
                consecutive_set = [(color, value), (color, value + 1), (color, value + 2), (color, value + 3)]
                consecutive_set[i] = "Joker"
                four_consecutive_sets_one_joker.append(consecutive_set)
        else:
            for i in range(3):
                consecutive_set = [(color, value), (color, value + 1), (color, value + 2), (color, value + 3)]
                consecutive_set[i] = "Joker"
                four_consecutive_sets_one_joker.append(consecutive_set)

In [23]:
four_consecutive_sets_two_joker = []

for color in Deck_colors:
    for value in Deck_values[:-3]:  # Iterate through values, stopping three before the end
        # Create a set of four consecutive cards of the same color
        if value == 1:
            for i in range(3):
                for j in range(i+1, 4):
                    consecutive_set = [(color, value), (color, value + 1), (color, value + 2), (color, value + 3)]
                    consecutive_set[i] = "Joker"
                    consecutive_set[j] = "Joker"
                    four_consecutive_sets_two_joker.append(consecutive_set)
        else:
            for i in range(2):
                for j in range(i+1, 3):
                    consecutive_set = [(color, value), (color, value + 1), (color, value + 2), (color, value + 3)]
                    consecutive_set[i] = "Joker"
                    consecutive_set[j] = "Joker"
                    four_consecutive_sets_two_joker.append(consecutive_set)

In [24]:
# Generating sets with five consecutive numbers, same color, and without Joker

five_consecutive_sets_no_joker = []

for color in Deck_colors:
    for value in Deck_values[:-4]:  # Iterate through values, stopping four before the end
        # Create a set of five consecutive cards of the same color
        consecutive_set = [(color, value), (color, value + 1), (color, value + 2), (color, value + 3), (color, value + 4)]
        five_consecutive_sets_no_joker.append(consecutive_set)

In [25]:
# Generating sets with five consecutive numbers, same color, and 1 Joker

five_consecutive_sets_one_joker = []

for color in Deck_colors:
    for value in Deck_values[:-4]:  # Iterate through values, stopping four before the end
        # Create a set of five consecutive cards of the same color
        if value == 1:
            for i in range(5):
                consecutive_set = [(color, value), (color, value + 1), (color, value + 2), (color, value + 3), (color, value + 4)]
                consecutive_set[i] = "Joker"
                five_consecutive_sets_one_joker.append(consecutive_set)
        else:
            for i in range(4):
                consecutive_set = [(color, value), (color, value + 1), (color, value + 2), (color, value + 3), (color, value + 4)]
                consecutive_set[i] = "Joker"
                five_consecutive_sets_one_joker.append(consecutive_set)

In [26]:
# Generating sets with five consecutive numbers, same color, and 2 Joker

five_consecutive_sets_two_joker = []

for color in Deck_colors:
    for value in Deck_values[:-4]:  # Iterate through values, stopping four before the end
        # Create a set of five consecutive cards of the same color
        if value == 1:
            for i in range(4):
                for j in range(i+1, 5):
                    consecutive_set = [(color, value), (color, value + 1), (color, value + 2), (color, value + 3), (color, value + 4)]
                    consecutive_set[i] = "Joker"
                    consecutive_set[j] = "Joker"
                    five_consecutive_sets_two_joker.append(consecutive_set)
        else:
            for i in range(3):
                for j in range(i+1, 4):
                    consecutive_set = [(color, value), (color, value + 1), (color, value + 2), (color, value + 3), (color, value + 4)]
                    consecutive_set[i] = "Joker"
                    consecutive_set[j] = "Joker"
                    five_consecutive_sets_two_joker.append(consecutive_set)

In [27]:
# Generating sets of three cards, each from a different color, without Jokers, and all cards having the same value

three_cards_different_colors_same_value = []

for value in Deck_values:
    # Iterate through combinations of three different colors
    for i in range(len(Deck_colors)):
        for j in range(i + 1, len(Deck_colors)):
            for k in range(j + 1, len(Deck_colors)):
                # Create a set with one card from each of the three different colors, all with the same value
                set_of_three = [(Deck_colors[i], value), (Deck_colors[j], value), (Deck_colors[k], value)]
                three_cards_different_colors_same_value.append(set_of_three)

In [28]:
# Generating sets of three cards, each from a different color, 1 Jokers, and all cards having the same value

three_cards_different_colors_1_joker = []

for value in Deck_values:
    # Iterate through combinations of three different colors
    for i in range(len(Deck_colors)):
        for j in range(i + 1, len(Deck_colors)):
            # Create a set with one card from each of the three different colors, all with the same value
            set_of_three = [(Deck_colors[i], value), (Deck_colors[j], value), "Joker"]
            three_cards_different_colors_1_joker.append(set_of_three)

In [29]:
four_cards_different_colors_same_value = []

for value in Deck_values:
    # Iterate through combinations of three different colors
    set_of_four = [(Deck_colors[0], value), (Deck_colors[1], value), (Deck_colors[2], value), (Deck_colors[3], value)]
    four_cards_different_colors_same_value.append(set_of_four)

In [30]:
# Generating sets of four cards, each from a different color, 1 Jokers, and all cards having the same value

four_cards_different_colors_1_joker = []

for value in Deck_values:
    # Iterate through combinations of three different colors
    for i in range(len(Deck_colors)):
        for j in range(i + 1, len(Deck_colors)):
            for k in range(j + 1, len(Deck_colors)):
                # Create a set with one card from each of the three different colors, all with the same value
                set_of_four = [(Deck_colors[i], value), (Deck_colors[j], value), (Deck_colors[k], value), "Joker"]
                four_cards_different_colors_1_joker.append(set_of_four)

In [31]:
# Generating sets of four cards, each from a different color, 2 Jokers, and all cards having the same value

four_cards_different_colors_2_joker = []

for value in Deck_values:
    # Iterate through combinations of three different colors
    for i in range(len(Deck_colors)):
        for j in range(i + 1, len(Deck_colors)):
            # Create a set with one card from each of the three different colors, all with the same value
            set_of_four = [(Deck_colors[i], value), (Deck_colors[j], value), "Joker", "Joker"]
            four_cards_different_colors_2_joker.append(set_of_four)

In [32]:
three_consecutives = three_consecutive_set_no_joker + three_consecutive_set_one_joker + three_consecutive_sets_with_two_jokers
four_consecutives = four_consecutive_sets_no_joker + four_consecutive_sets_one_joker + four_consecutive_sets_two_joker
five_consecutives = five_consecutive_sets_no_joker + five_consecutive_sets_one_joker + five_consecutive_sets_two_joker
three_same = three_cards_different_colors_same_value + three_cards_different_colors_1_joker
four_same = four_cards_different_colors_same_value + four_cards_different_colors_1_joker + four_cards_different_colors_2_joker
all_sets = three_consecutives + four_consecutives + five_consecutives + three_same + four_same

In [33]:
len(all_sets)

1173

## Decision variables

In [12]:
S_reg = model.addVars(Deck_types, Deck_values, J, vtype = GRB.BINARY, 
                  name = ["tile "+ dt + " " + str(dv) + " is in set " + str(j+1) 
                          for dt in Deck_types for dv in Deck_values for j in range(J)])
S_jok = model.addVars(Joker, Joker_values, J, vtype = GRB.BINARY, 
                      name = ["tile Joker " + dv + " is in set " + str(j+1) 
                              for dt in Joker for dv in Joker_values for j in range(J)])

In [13]:
T_reg = model.addVars(Deck_types, Deck_values, lb = 0, ub = 2, vtype = GRB.INTEGER, 
                  name = ["times of tile " + dt + " " + str(dv) + " on the table" for dt in Deck_types for dv in Deck_values])
T_jok = model.addVars(Joker, Joker_values, lb = 0, ub = 2, vtype = GRB.INTEGER, 
                  name = ["times of tile Joker " + dv + "on the table" for dt in Joker for dv in Joker_values])

In [14]:
R_reg = model.addVars(Deck_types, Deck_values, lb = 0, ub = 2, vtype = GRB.INTEGER, 
                   name = ["times of tile " + dt + " " + str(dv) +" on your rack" for dt in Deck_types for dv in Deck_values])
R_jok = model.addVars(Joker, Joker_values, lb = 0, ub = 2, vtype = GRB.INTEGER, 
                  name = ["times of tile Joker" + dv +"on your rack" for dt in Joker for dv in Joker_values])

In [16]:
Y_reg = model.addVars(Deck_types, Deck_values, lb = 0, ub = 2, vtype = GRB.INTEGER, 
                  name = ["times of tile "+ dt + " " + str(dv) +" can be placed from your rack onto the table" 
                          for dt in Deck_types for dv in Deck_values])
Y_jok = model.addVars(Joker, Joker_values, lb = 0, ub = 2, vtype = GRB.INTEGER, 
                  name = ["times of tile Joker " + dv +" can be placed from your rack onto the table" 
                          for dt in Joker for dv in Joker_values])

In [72]:
X = model.addVars(J, lb = 0, ub = 2, vtype = GRB.INTEGER, 
                  name = ["times of set "+str(all_sets[j])+" can be placed onto the table" for j in range(J)])

In [17]:
# W = model.addVars(J, lb = 0, ub = 2, vtype = GRB.INTEGER, 
#                   name = ["times of set "+str(j+1)+" on the table" for j in range(J)])
# M = 40
# Z = model.addVars(J, lb = 0, ub = 2, vtype = GRB.INTEGER, 
#                   name = ["times of set "+str(j+1)+" occurs in the old and in the new solution" for j in range(J)])

## Objective function

In [34]:
# model.setObjective(quicksum(Value[i] * Y[i] for i in range(I)) + 1/M * quicksum(Z[j] for j in range(J)), GRB.MAXIMIZE)
exp1 = quicksum(Value[value-1] * Y_reg[types, value] for types in Deck_types for value in Deck_values)
exp2 = quicksum(Y_jok) * 30
model.setObjective(exp1 + exp2, GRB.MAXIMIZE)

## Constraints

In [35]:
# how many times a tile is on the table
model.addConstrs(quicksum(S_reg[types, value, j] * X[j] for j in range(J)) == T_reg[types, value] + Y_reg[types, value] 
                 for types in Deck_types for value in Deck_values)
model.addConstr(quicksum(quicksum(S_jok) * X[j] for j in range(J)) == quicksum(T_jok) + quicksum(Y_jok))
#model.addConstrs(quicksum(S[i, j]*X[j] for j in range(J)) == T[i] + Y[i] for i in range(I))
model.update()

In [36]:
# number of times on the rack <= number of times moved from rack to table
model.addConstrs(Y_reg[dt, dv] <= R_reg[dt, dv] for dt in Deck_types for dv in Deck_values)
model.addConstrs(Y_jok["Joker", dv] <= R_jok["Joker", dv] for dv in Joker_values)

{'J1': <gurobi.Constr *Awaiting Model Update*>,
 'J2': <gurobi.Constr *Awaiting Model Update*>}

In [39]:
# number of times a set in a solution <= number of times a set being placed on the table
# model.addConstrs(Z[j] <= X[j] for j in range(J))
# number of times a set in a solution <= number of times a set on the table
# model.addConstrs(Z[j] <= W[j] for j in range(J))

In [52]:
all_sets.index([('Red', 11), ('Red', 12), ('Red', 13)])

41

In [78]:
model.addConstr(S_reg["Black1", 1, 188] == 1) # tiles in set
model.addConstr(T_reg["Black1", 1] == T_reg["Black1", 1] + 1) # times of tile on the table
model.addConstr(Y_reg["Black1", 1] == Y_reg["Black1", 1] - 1) # times of tile can be placed from rack to table
# model.addConstr(R_reg["Black1", 1] == R_reg["Black1", 1] - 1) # times of tile on your rack

model.addConstr(S_reg["Black1", 2, 188] == 1)
model.addConstr(T_reg["Black1", 2] == T_reg["Black1", 2] + 1)
model.addConstr(Y_reg["Black1", 2] == Y_reg["Black1", 2] - 1)
# model.addConstr(R_reg["Black1", 2] == R_reg["Black1", 2] - 1)

model.addConstr(S_reg["Black1", 3, 188] == 1)
model.addConstr(T_reg["Black1", 3] == T_reg["Black1", 3] + 1)
model.addConstr(Y_reg["Black1", 3] == Y_reg["Black1", 3] - 1)
# model.addConstr(R_reg["Black1", 3] == R_reg["Black1", 3] - 1)

model.addConstr(S_reg["Black1", 4, 188] == 1)
model.addConstr(T_reg["Black1", 4] == T_reg["Black1", 4] + 1)
model.addConstr(Y_reg["Black1", 4] == Y_reg["Black1", 4] - 1)
# model.addConstr(R_reg["Black1", 4] == R_reg["Black1", 4] - 1)

model.addConstr(S_reg["Black1", 10, 36] == 1)
model.addConstr(T_reg["Black1", 10] == T_reg["Black1", 10] + 1)
model.addConstr(Y_reg["Black1", 10] == Y_reg["Black1", 10] - 1)
# model.addConstr(R_reg["Black1", 10] == R_reg["Black1", 10] - 1)

model.addConstr(S_reg["Black1", 11, 36] == 1)
model.addConstr(T_reg["Black1", 11] == T_reg["Black1", 11] + 1)
model.addConstr(Y_reg["Black1", 11] == Y_reg["Black1", 11] - 1)
# model.addConstr(R_reg["Black1", 11] == R_reg["Black1", 11] - 1)

model.addConstr(S_reg["Black1", 12, 36] == 1)
model.addConstr(T_reg["Black1", 12] == T_reg["Black1", 12] + 1)
model.addConstr(Y_reg["Black1", 12] == Y_reg["Black1", 12] - 1)
# model.addConstr(R_reg["Black1", 12] == R_reg["Black1", 12] - 1)

model.addConstr(S_reg["Orange1", 6, 23] == 1)
model.addConstr(T_reg["Orange1", 6] == T_reg["Orange1", 6] + 1)
model.addConstr(Y_reg["Orange1", 6] == Y_reg["Orange1", 6] - 1)
# model.addConstr(R_reg["Orange1", 6] == R_reg["Orange1", 6] - 1)

model.addConstr(S_reg["Orange1", 7, 23] == 1)
model.addConstr(T_reg["Orange1", 7] == T_reg["Orange1", 7] + 1)
model.addConstr(Y_reg["Orange1", 7] == Y_reg["Orange1", 7] - 1)
# model.addConstr(R_reg["Orange1", 7] == R_reg["Orange1", 7] - 1)

model.addConstr(S_reg["Orange1", 8, 23] == 1)
model.addConstr(T_reg["Orange1", 8] == T_reg["Orange1", 8] + 1)
model.addConstr(Y_reg["Orange1", 8] == Y_reg["Orange1", 8] - 1)
# model.addConstr(R_reg["Orange1", 8] == R_reg["Orange1", 8] - 1)

model.addConstr(S_reg["Orange1", 9, 519] == 1)
model.addConstr(T_reg["Orange1", 9] == T_reg["Orange1", 9] + 1)
model.addConstr(Y_reg["Orange1", 9] == Y_reg["Orange1", 9] - 1)
# model.addConstr(R_reg["Orange1", 9] == R_reg["Orange1", 9] - 1)

model.addConstr(S_reg["Orange1", 10, 519] == 1)
model.addConstr(T_reg["Orange1", 10] == T_reg["Orange1", 10] + 1)
model.addConstr(Y_reg["Orange1", 10] == Y_reg["Orange1", 10] - 1)
# model.addConstr(R_reg["Orange1", 10] == R_reg["Orange1", 10] - 1)

model.addConstr(S_reg["Orange1", 11, 519] == 1)
model.addConstr(T_reg["Orange1", 11] == T_reg["Orange1", 11] + 1)
model.addConstr(Y_reg["Orange1", 11] == Y_reg["Orange1", 11] - 1)
# model.addConstr(R_reg["Orange1", 11] == R_reg["Orange1", 11] - 1)

model.addConstr(S_reg["Orange1", 12, 519] == 1)
model.addConstr(T_reg["Orange1", 12] == T_reg["Orange1", 12] + 1)
model.addConstr(Y_reg["Orange1", 12] == Y_reg["Orange1", 12] - 1)
# model.addConstr(R_reg["Orange1", 12] == R_reg["Orange1", 12] - 1)

model.addConstr(S_reg["Orange1", 13, 519] == 1)
model.addConstr(T_reg["Orange1", 13] == T_reg["Orange1", 13] + 1)
model.addConstr(Y_reg["Orange1", 13] == Y_reg["Orange1", 13] - 1)
# model.addConstr(R_reg["Orange1", 13] == R_reg["Orange1", 13] - 1)

model.addConstr(S_reg["Blue1", 7, 26] == 1)
model.addConstr(T_reg["Blue1", 7] == T_reg["Blue1", 7] + 1)
model.addConstr(Y_reg["Blue1", 7] == Y_reg["Blue1", 7] - 1)
# model.addConstr(R_reg["Blue1", 7] == R_reg["Blue1", 7] - 1)

model.addConstr(S_reg["Blue1", 8, 26] == 1)
model.addConstr(T_reg["Blue1", 8] == T_reg["Blue1", 8] + 1)
model.addConstr(Y_reg["Blue1", 8] == Y_reg["Blue1", 8] - 1)
# model.addConstr(R_reg["Blue1", 8] == R_reg["Blue1", 8] - 1)

model.addConstr(S_reg["Blue1", 9, 26] == 1)
model.addConstr(T_reg["Blue1", 9] == T_reg["Blue1", 9] + 1)
model.addConstr(Y_reg["Blue1", 9] == Y_reg["Blue1", 9] - 1)
# model.addConstr(R_reg["Blue1", 9] == R_reg["Blue1", 9] - 1)

model.addConstr(S_reg["Blue1", 10, 38] == 1)
model.addConstr(T_reg["Blue1", 10] == T_reg["Blue1", 10] + 1)
model.addConstr(Y_reg["Blue1", 10] == Y_reg["Blue1", 10] - 1)
# model.addConstr(R_reg["Blue1", 10] == R_reg["Blue1", 10] - 1)

model.addConstr(S_reg["Blue1", 11, 38] == 1)
model.addConstr(T_reg["Blue1", 11] == T_reg["Blue1", 11] + 1)
model.addConstr(Y_reg["Blue1", 11] == Y_reg["Blue1", 11] - 1)
# model.addConstr(R_reg["Blue1", 11] == R_reg["Blue1", 11] - 1)

model.addConstr(S_reg["Blue1", 12, 38] == 1)
model.addConstr(T_reg["Blue1", 12] == T_reg["Blue1", 12] + 1)
model.addConstr(Y_reg["Blue1", 12] == Y_reg["Blue1", 12] - 1)
# model.addConstr(R_reg["Blue1", 12] == R_reg["Blue1", 12] - 1)

model.addConstr(S_reg["Red1", 11, 41] == 1)
model.addConstr(T_reg["Red1", 11] == T_reg["Red1", 11] + 1)
model.addConstr(Y_reg["Red1", 11] == Y_reg["Red1", 11] - 1)
# model.addConstr(R_reg["Red1", 11] == R_reg["Red1", 11] - 1)

model.addConstr(S_reg["Red1", 12, 41] == 1)
model.addConstr(T_reg["Red1", 12] == T_reg["Red1", 12] + 1)
model.addConstr(Y_reg["Red1", 12] == Y_reg["Red1", 12] - 1)
# model.addConstr(R_reg["Red1", 12] == R_reg["Red1", 12] - 1)

model.addConstr(S_reg["Red1", 13, 41] == 1)
model.addConstr(T_reg["Red1", 13] == T_reg["Red1", 13] + 1)
model.addConstr(Y_reg["Red1", 13] == Y_reg["Red1", 13] - 1)
# model.addConstr(R_reg["Red1", 13] == R_reg["Red1", 13] - 1)

for i in [188, 36, 23, 519, 26, 38, 41]:
    model.addConstr(X[i] == X[i] -1)

## Optimize

In [79]:
model.optimize()
model.status

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i9-9880H CPU @ 2.30GHz
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 288 rows, 129455 columns and 260 nonzeros
Model fingerprint: 0xe0699984
Model has 105 quadratic constraints
Variable types: 0 continuous, 129455 integer (124444 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 3e+01]
  Bounds range     [1e+00, 2e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.04s

Explored 0 nodes (0 simplex iterations) in 0.07 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 0
No other solutions better than -1e+100

Model is infeasible
Best objective -, best bound -, gap -


3

In [13]:
model.ObjVal

846.6999999999466

In [16]:
# for v in model.getVars():
#     if v.x != 0:
#         print (v.varName, v.x)

##### Some thoughts

decision variables maybe should be separated by color

interactive so that have enough tile on the table to solve and optimize

after solving using ILP, show the solution in plain language, also show how many extra steps it will need to finish the game